<script defer src="https://cdn.jsdelivr.net/npm/katex@0.16.11/dist/katex.min.js" integrity="sha384-7zkQWkzuo3B5mTepMUcHkMB5jZaolc2xDwL6VFqjFALcbeS9Ggm/Yr2r3Dy4lfFg" crossorigin="anonymous"></script>
# Tensor 张量

张量是一种特殊的数据结构，与向量和矩阵非常相似（向量是 1 阶张量，而矩阵是 2 阶张量）。在 PyTorch 中，我们使用张量来编码模型的输入和输出，以及模型的参数

>张量类似于 [NumPy](https://numpy.com.cn/) 的 ndarray，但张量可以在 GPU 或其他硬件加速器上运行。事实上，张量和 NumPy 数组通常可以共享相同的底层内存，从而无需复制数据（请参阅 [与 NumPy 的桥接](https://pytorch.ac.cn/tutorials/beginner/blitz/tensor_tutorial.html#bridge-to-np-label)）

In [37]:
import torch
import numpy as np

## 初始化张量

初始化张量有以下多种方式

In [38]:
# 直接从 Python 列表初始化
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
x_data

tensor([[1, 2],
        [3, 4]])

In [39]:
# 从 Numpy 多维数组创建张量
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
x_np

tensor([[1, 2],
        [3, 4]])

In [40]:
# 从另一个张量初始化
# 除非显式覆盖，否则新张量将保留参数张量的属性（形状、数据类型）
# xxx_like 方法会创建一个形状和参数张量相同的张量
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.9947, 0.7799],
        [0.2547, 0.1404]]) 



In [41]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.0239, 0.3933, 0.6001],
        [0.5332, 0.9554, 0.0595]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


### 随机初始化

pytorch 支持符合多种分布的随机初始化

In [78]:
# 创建基础张量
base = torch.empty(3,3).uniform_(0,1)
base

tensor([[0.8533, 0.7253, 0.9860],
        [0.5154, 0.9288, 0.9403],
        [0.3603, 0.4420, 0.4256]])

>torch.empty 创建的张量中的元素不会初始化为特定的值。相反，这些元素会包含内存中当前的残留值，这些值是未定义的，可能是任意的浮点数

#### 伯努利分布

伯努利分布（Bernoulli distribution）是一种离散型概率分布，用于描述只有两种可能结果的单次随机试验或实验。这两种结果通常被称为“成功”和“失败”，在数学表达中分别用 1 和 0 来表示。伯努利分布是二项分布的特例，当进行一次伯努利试验时，其结果服从伯努利分布

In [92]:
# 伯努利分布（其中原始张量的每一位都代表其元素在伯努利事件中为 1 的概率）
torch.bernoulli(base)

tensor([[1., 1., 1.],
        [0., 1., 1.],
        [0., 0., 1.]])

#### 多项式分布

多项式分布（Multinomial Distribution）是一种离散概率分布，用于描述在固定次数的独立实验中，每个实验有多个可能的结果，并且每个实验的结果互斥时，每个可能结果出现次数的概率分布

In [166]:
# 多项式分布
# 定义每个类别的权重，对应于每个结果的概率
weights = torch.tensor([15,20,25,40], dtype=torch.float)
# 执行抽样, replacement 代表抽烟是带替换的，即一个样本可以在结果中出现多次
torch.multinomial(weights, 10, replacement=True)

tensor([3, 2, 3, 3, 3, 1, 1, 2, 2, 3])

#### 高斯分布

高斯分布（Gaussian distribution），是统计学与概率论中最常见的概率分布之一。其图形是一个对称的钟形曲线，被称为正态分布曲线或高斯钟形曲线

In [167]:
# 高斯分布（每一维的均值和方差分别由张量参数决定）
torch.normal(mean=torch.arange(1., 11.), std=torch.arange(1, 0, -0.1))

tensor([1.0797, 0.4763, 3.3771, 2.1580, 4.4411, 5.6283, 6.6492, 7.9054, 8.8766,
        9.8960])

In [174]:
# 高斯分布（固定均值和方差，生成一个高斯数）
torch.normal(mean=torch.tensor(0.0), std=torch.tensor(1.0))

tensor(-0.9422)

#### 泊松分布

泊松分布（Poisson Distribution）是一种离散概率分布，用于描述在固定时间间隔或固定空间内发生某事件的次数的概率。它经常用于描述稀有事件的计数数据，例如电话呼叫中心接到的呼叫次数、某网站在特定时间内的访问次数、放射性物质发射粒子的数量等，当事件的平均发生率较大时，泊松分布将逼近高斯分布

In [222]:
# 泊松分布
# 生成 4 维向量，其中每个元素都是 0 到 10 之间的随机数，这些值将作为泊松分布是发生率参数
rates = torch.rand(4) * 10
torch.poisson(rates)

tensor([6., 4., 6., 5.])

#### 均匀分布

均匀分布（Uniform Distribution）是一种连续概率分布，它在某个区间内的任意子区间上发生的概率是相等的。也就是说，这个区间内的任何值都有相同的概率被选中

In [227]:
# 均匀分布
torch.rand(4)

tensor([0.3807, 0.8422, 0.8857, 0.6314])

>`shape` 是一个张量形状元组。在以上脚本中，它决定了输出张量的形状，通常在神经网络计算中，张量的第一个维度是 batch size（批次大小）

## 张量的属性

张量属性描述了它们的形状、数据类型以及存储它们的设备

In [42]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


## 张量上的运算

张量支持超过 100 种运算，包括算术、线性代数、矩阵操作（转置、索引、切片）、采样等等

## 张量的方法

| 操作                   | 注释                                                                                                         |
|:---------------------|:-----------------------------------------------------------------------------------------------------------|
| `is_tensor`          | 如果 obj 是 PyTorch 张量，则返回 True                                                                               |
| `is_storage`         | 如果 obj 是 PyTorch 存储对象，则返回 True                                                                             |
| `is_complex`         | 如果 `input` 的数据类型是复数数据类型，即 `torch.complex64` 和 `torch.complex128` 中的一种，则返回 True                             |
| `is_conj`            | 如果 `input` 是共轭张量，即其共轭位设置为 True，则返回 True                                                                    |
| `is_floating_point`  | 如果 `input` 的数据类型是浮点数据类型，即 `torch.float64`、`torch.float32`、`torch.float16` 和 `torch.bfloat16` 中的一种，则返回 True |
| `is_nonzero`         | 如果 `input` 是一个单元素张量，并且在类型转换后不等于零，则返回 True                                                                  |
| `set_default_dtype`  | 将默认浮点数据类型设置为 `d`                                                                                           |
| `get_default_dtype`  | 获取当前默认浮点 `torch.dtype`                                                                                     |
| `set_default_device` | 设置要在 `device` 上分配的默认 `torch.Tensor`                                                                        |
| `get_default_device` | 获取要在 `device` 上分配的默认 `torch.Tensor`                                                                        |
| `numel`              | 返回 `input` 张量中的元素总数                                                                                        |
| `set_printoptions`   | 设置打印选项                                                                                                     |
| `set_flush_denormal` | 在 CPU 上禁用非规格化浮点数                                                                                           |

默认情况下，张量是在 CPU 上创建的。我们需要使用 .to 方法（在检查 GPU 可用性之后）将张量显式移动到 GPU。

>跨设备复制大型张量在时间和内存方面可能代价高昂

In [290]:
# 张量算术运算（以向量、矩阵，的加法、乘法运算为例）
a = torch.tensor([1,2,3])
b = torch.tensor([4,5,6])
a_mat = torch.tensor([[1,2,3],[3,2,1],[3,1,2]])
b_mat = torch.tensor([[4,5,6],[6,5,4],[6,4,5]])

In [44]:
# 向量加法
a + b

tensor([5, 7, 9])

In [45]:
# 矩阵加向量
a_mat + b

tensor([[5, 7, 9],
        [7, 7, 7],
        [7, 6, 8]])

In [46]:
# 矩阵加法
a_mat + b_mat

tensor([[5, 7, 9],
        [9, 7, 5],
        [9, 5, 7]])

In [49]:
# 向量点积
a * b

tensor([ 4, 10, 18])

In [48]:
# 向量叉积（叉积仅适用于三维空间中的向量）
torch.linalg.cross(a, b)

tensor([-3,  6, -3])

In [249]:
# 矩阵乘法
a_mat @ b_mat

tensor([[34, 27, 29],
        [30, 29, 31],
        [30, 28, 32]])

In [53]:
# 逐元素指数运算
torch.exp(a)

tensor([ 2.7183,  7.3891, 20.0855])

In [55]:
# 逐元素对数运算
torch.log(a)

tensor([0.0000, 0.6931, 1.0986])

### 其他运算

In [240]:
# 连接张量（在第一个维度）
torch.cat([a_mat, b_mat], dim=0)

tensor([[1, 2, 3],
        [3, 2, 1],
        [3, 1, 2],
        [4, 5, 6],
        [6, 5, 4],
        [6, 4, 5]])

In [241]:
# 连接张量（在第二个维度）
torch.cat([a_mat, b_mat], dim=1)

tensor([[1, 2, 3, 4, 5, 6],
        [3, 2, 1, 6, 5, 4],
        [3, 1, 2, 6, 4, 5]])

In [245]:
# 转置
a_mat.T

tensor([[1, 3, 3],
        [2, 2, 1],
        [3, 1, 2]])

## 张量和 Numpy 数组的转换

CPU 上的张量和 NumPy 数组可以共享它们的底层内存位置，更改其中一个将更改另一个

In [291]:
# 张量到 python 数组
a_np = a_mat.numpy()
a_np

array([[1, 2, 3],
       [3, 2, 1],
       [3, 1, 2]])

In [292]:
# 张量的变化会反映在 ndarray 上，因为引用相同
a_mat.add_(1)
a_np

array([[2, 3, 4],
       [4, 3, 2],
       [4, 2, 3]])

In [293]:
# ndarray 到张量
torch.from_numpy(np.ones(5))

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)